# Data processing and functions for STA160 Midterm 1  
#### Project Link: https://www.kaggle.com/datasets/alexteboul/heart-disease-health-indicators-dataset
#### Author: Xing Yang Lan

In [1]:
import math
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from timeit import default_timer
from numpy import unique
from numpy import sqrt, dot, array, diagonal, mean, transpose
from numpy import transpose, diag, dot
from numpy.linalg import svd, inv, qr


from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor 
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
random.seed(1)

In [3]:
df = pd.read_csv("heart_disease_health_indicators_BRFSS2015.csv")

,HeartDiseaseorAttack,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,Diabetes,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0


In [4]:
colNames = df.columns
print(f"There are {len(colNames)} variables,\n{colNames}")

There are 22 variables,
Index(['HeartDiseaseorAttack', 'HighBP', 'HighChol', 'CholCheck', 'BMI',
       'Smoker', 'Stroke', 'Diabetes', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth',
       'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education',
       'Income'],
      dtype='object')


In [5]:
# kaggle: "229,787 respondents do not have/have not had heart disease while 23,893 have had heart disease."
print(f"The first column is the Y/Response Variable {colNames[0]}")
# check that 0 means no heart disease and 1 means heart disease
print(list(df["HeartDiseaseorAttack"]).count(0), list(df["HeartDiseaseorAttack"]).count(1))

The first column is the Y/Response Variable HeartDiseaseorAttack
229787 23893


In [6]:
dfX = df.copy().drop(columns=["HeartDiseaseorAttack"])
# dfX is a pandas DataFrame now (pandas.core.frame.DataFrame)

dfY = df["HeartDiseaseorAttack"]
# dfY is a series now (pandas.core.series.Series)

X = array(dfX)
# X is a "matrix" list ("np.ndarray") of lists ("np.ndarray") where each list inside is the 21 Predictor Variables
Y = array(dfY).reshape(len(dfY), 1)
# Y is a list ("np.ndarray") of lists ("np.ndarray") that is basically [[0],[0],[0],[0],[0]...]'

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=1)

In [7]:
# This is a check. This should be "True" ; the first 60 observations of the first variable of X is the first response variable
print(all(X[0:60,0] == df[colNames[1]][:60]))

True


# Baseline Model

In [8]:
def check(vec):
    # It will use np.ravel() on pred/Y if it is a list of lists
    # ravel() basically turns [[0],[0],[0],[0]] into [0,0,0,0] so you don't get the annoying warning sklearn messages
    if type(vec[0]) not in (int, float, np.float64):
        return np.ravel(vec)
    return vec

def accuracy(Y_pr, Y_ts):
    # It returns the percent correct for your predictions
    Y_pr, Y_ts = check(Y_pr), check(Y_ts)
    return sum([1 if pr == ts else 0 for pr,ts in zip(Y_pr, Y_ts)]) / len(Y_pr)

def confusion(Y_pr, Y_ts):
    # returns a confusion table
    Y_pr, Y_ts = check(Y_pr), check(Y_ts)
    TP, FP, TN, FN = 0, 0, 0, 0
    dd = {(1,1):TP, (1,0):FP, (0,1):FN, (0,0):TN}
    for pr, ts in zip(Y_pr, Y_ts):
        dd[(pr,ts)] += 1
    return {"TP": dd[(1,1)], "FP": dd[(1,0)], "FN": dd[(0,1)], "TN": dd[(0,0)]}

def convertToBinary(Y_pr, cuttoffPercentile=85):
    # an arbituary function that allows you to use regressors
    cutoff = np.percentile(Y_pr, cuttoffPercentile)
    return [1.0 if pr > cutoff else 0.0 for pr in Y_pr]

# Run a model. Literally any model that does not need to one-hot-encode
def runModel(model, __X, __Y, cuttoffPercentile=90):
    __X_train, __X_test, __Y_train, __Y_test = train_test_split(__X, __Y, test_size=0.30, random_state=1)
    model.fit(__X_train, np.ravel(__Y_train))
    modelPred = model.predict(__X_test)
    if any([0.000001 < pr < 0.99999 or pr < -0.000001 for pr in modelPred]):
        modelPred = convertToBinary(modelPred, cuttoffPercentile)
    print("Accuracy:", accuracy(modelPred, __Y_test))
    print("MSE:", mean_squared_error(modelPred, __Y_test))
    print("Confusion:", confusion(modelPred, __Y_test))
    return model


def runModelGBR(model, __X, __Y, cuttoffPercentile=85):
    __X_train, __X_test, __Y_train, __Y_test = train_test_split(__X, __Y, test_size=0.30, random_state=1)
    model.fit(__X_train, np.ravel(__Y_train))
    modelPredOG = model.predict(__X_test)
    for cutoff in (85, 90, 94):
        modelPred = convertToBinary(modelPredOG, cutoff)
        print("Accuracy:", accuracy(modelPred, __Y_test))
        print("MSE:", mean_squared_error(modelPred, __Y_test))
        print("Confusion:", confusion(modelPred, __Y_test))
    return model

def runFittedModel(model, __X, __Y, cuttoffPercentile=90):
    __X_train, __X_test, __Y_train, __Y_test = train_test_split(__X, __Y, test_size=0.30, random_state=1)
    modelPred = model.predict(__X_test)
    if any([0.000001 < pr < 0.99999 or pr < -0.000001 for pr in modelPred]):
        modelPred = convertToBinary(modelPred, cuttoffPercentile)
    conf = confusion(modelPred, __Y_test)
    sens = conf["TP"] / (conf["TP"] + conf["FN"])
    spec = conf["TN"] / (conf["FP"] + conf["TN"])
    print(f"{model=}")
    print(f"{accuracy(modelPred, __Y_test)=}")
    print(f"{mean_squared_error(modelPred, __Y_test)=}")
    print(f"Confusion: {conf=}\n")
    print(f"Sensitivity: {sens}")
    print(f"Specificity: {spec}")

In [9]:
# All: GenHlth, MenHlth, PhysHlth, Age, Education, Diabetes, BMI
# Unused: "GenHlth", "Education", "Diabetes", 

# reduces continuous columns to categories of values in rangles
#### after training multiple MLPs, this does not help with overfitting
def refactorColsEncode():
    __dfX, cols, cache = dfX.copy(), ["MentHlth", "PhysHlth", "Age", "BMI"], []
    for col in cols:
        cache.append(__dfX[col])
    __dfX.drop(columns=cols)
    for i in range(len(cols)):
        __dfX[cols[i]] = array(cache[i]) // len(list(unique(list(dfX[col])))) // 6
    return __dfX

In [12]:
encoder = OneHotEncoder()

X_enc = encoder.fit(X).transform(X).toarray()

In [25]:
# percent of ppl with heart disease
list(Y_test.ravel()).count(1) / (0.01*len(Y_test))

9.418690213392201

In [15]:
# 2015 ppl with heart disease/2016 pop(2015 report, 2016 pop)
121.5/(0.78*323.1)

0.4821084208270837

In [16]:
Lasso1 = runModel(Lasso(alpha=0.000001), X, Y, 90)

Accuracy: 0.8802034058656575
MSE: 0.11979659413434247
Confusion: {'TP': 2831, 'FP': 4780, 'FN': 4337, 'TN': 64156}


In [18]:
# X_enc is now one hot enc
print("\nGBR_NE460_MD3:")
GBR_NE460_MD3 = GradientBoostingRegressor(n_estimators=450, max_depth=3, random_state=1)
GBR_NE460_MD3 = runModelGBR(GBR_NE460_MD3, X_enc, Y)


GBR_NE460_MD3:
Accuracy: 0.8573793755912961
MSE: 0.14262062440870388
Confusion: {'TP': 3865, 'FP': 7551, 'FN': 3303, 'TN': 61385}
Accuracy: 0.8842504993167245
MSE: 0.11574950068327551
Confusion: {'TP': 2985, 'FP': 4626, 'FN': 4183, 'TN': 64310}
Accuracy: 0.9005177126038053
MSE: 0.09948228739619468
Confusion: {'TP': 2082, 'FP': 2485, 'FN': 5086, 'TN': 66451}


In [20]:
"""
5/9, 4:32PM, rerunning the Gradient boosted
X_enc is completely X_enc, which was how it was og run
"""

'\n5/9, 4:32PM, rerunning the Gradient boosted\nX_enc is completely X_enc, which was how it was og run\n'

In [23]:
for __cutoff in (94, 9, 85, 75, 48.2):
    print(f"{__cutoff=}")
    runFittedModel(GBR_NE460_MD3, X_enc, Y, __cutoff)
    print("\n")

__cutoff=94
model=GradientBoostingRegressor(n_estimators=450, random_state=1)
accuracy(modelPred, __Y_test)=0.9005177126038053
mean_squared_error(modelPred, __Y_test)=0.09948228739619468
Confusion: conf={'TP': 2082, 'FP': 2485, 'FN': 5086, 'TN': 66451}

Sensitivity: 0.2904575892857143
Specificity: 0.9639520714865962


__cutoff=90
model=GradientBoostingRegressor(n_estimators=450, random_state=1)
accuracy(modelPred, __Y_test)=0.8842504993167245
mean_squared_error(modelPred, __Y_test)=0.11574950068327551
Confusion: conf={'TP': 2985, 'FP': 4626, 'FN': 4183, 'TN': 64310}

Sensitivity: 0.4164341517857143
Specificity: 0.9328942787513056


__cutoff=85
model=GradientBoostingRegressor(n_estimators=450, random_state=1)
accuracy(modelPred, __Y_test)=0.8573793755912961
mean_squared_error(modelPred, __Y_test)=0.14262062440870388
Confusion: conf={'TP': 3865, 'FP': 7551, 'FN': 3303, 'TN': 61385}

Sensitivity: 0.5392020089285714
Specificity: 0.8904636184286875


__cutoff=75
model=GradientBoostingRegres

In [27]:
runFittedModel(GBR_NE460_MD3, X_enc, Y, 51.8)

model=GradientBoostingRegressor(n_estimators=450, random_state=1)
accuracy(modelPred, __Y_test)=0.5984442342058236
mean_squared_error(modelPred, __Y_test)=0.4015557657941764
Confusion: conf={'TP': 6645, 'FP': 30037, 'FN': 523, 'TN': 38899}

Sensitivity: 0.9270368303571429
Specificity: 0.5642770105605199


In [26]:
runFittedModel(GBR_NE460_MD3, X_enc, Y, 90.58)

model=GradientBoostingRegressor(n_estimators=450, random_state=1)
accuracy(modelPred, __Y_test)=0.8870361610427836
mean_squared_error(modelPred, __Y_test)=0.11296383895721644
Confusion: conf={'TP': 2870, 'FP': 4299, 'FN': 4298, 'TN': 64637}

Sensitivity: 0.400390625
Specificity: 0.9376378089822444
